In [1]:
# %cd /home/ubu-admin/Developer/tensorflow-wavelets
%cd /workspaces/OpenDVCW
import os
import load
import OpenDVCW
import tensorflow as tf
from math import log10, sqrt
import cv2
import numpy as np
from train import TrainOpenDVCW

/workspaces/OpenDVCW


In [2]:
def PSNR(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    if(mse == 0):  # MSE is zero means no noise is present in the signal .
                  # Therefore PSNR have no importance.
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

In [3]:
def compare(original_path, compressed_path, binary_path):
    original = cv2.imread(original_path)
    compressed = cv2.imread(compressed_path)
    bin_size = os.path.getsize(binary_path)
    value = PSNR(original, compressed)
    print("bin size: ", bin_size , "psnr: ", value, "BPP: ", bin_size/(240*240*3))

In [4]:
BATCH_SIZE = 1
Height = 240
Width = 240
lmbda = 256
# path = load.load_path_n("folder_cloud_test.npy", 0)
path = "/mnt/WindowsDev/DataSets/Beauty_1920x1080_120fps_420_8bit_YUV_RAW/"
p_frame_out_bin = "Test_com/dvcw/p_frame_dvcw.bin"
out_decom = "Test_com/dvcw/frame1.png"
i_on_test = "Test_com/frame0.png"
p_on_test = "Test_com/frame1.png"

i_frame = path + 'im1' + '.png'
p_frame = path + 'im2' + '.png'
print(i_frame)
print(p_frame)
OpenDVCW.write_png(p_on_test, OpenDVCW.read_png_crop(p_frame, 240, 240))
OpenDVCW.write_png(i_on_test, OpenDVCW.read_png_crop(i_frame, 240, 240))

/mnt/WindowsDev/DataSets/Beauty_1920x1080_120fps_420_8bit_YUV_RAW/im1.png
/mnt/WindowsDev/DataSets/Beauty_1920x1080_120fps_420_8bit_YUV_RAW/im2.png


2022-05-28 14:40:50.090247: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-28 14:40:50.102935: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-28 14:40:50.103632: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-28 14:40:50.105965: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [5]:
def test(model, p_on_test, out_decom, p_frame_out_bin):
    OpenDVCW.compress(model, i_frame, p_frame, p_frame_out_bin, 240, 240)
    OpenDVCW.decompress(model, i_frame, p_frame_out_bin, out_decom, 240, 240)
    original = cv2.imread(p_on_test)
    compressed = cv2.imread(out_decom)
    bin_size = os.path.getsize(p_frame_out_bin)
    value = PSNR(original, compressed)
    print("bin size: ", bin_size , "psnr: ", value, "BPP: ", bin_size/(240*240*3))

In [6]:

print("Loading model from saved model")
model_2048 = tf.keras.models.load_model('model_save_checkpoints_wavelets_haar_Lmbd_2048_epcs_1500_I_QP_27_240x240_CosineDecay_20220506-205303')
model_4096 = tf.keras.models.load_model('model_save_checkpoints_wavelets_haar_Lmbd_4096_epcs_1500_I_QP_27_240x240_CosineDecay_20220503-160827')
model_8192 = tf.keras.models.load_model('model_save_checkpoints_wavelets_haar_Lmbd_8192_epcs_1500_I_QP_27_240x240_CosineDecay_20220504-144848')
model_16384 = tf.keras.models.load_model('model_save_checkpoints_wavelets_haar_Lmbd_16384_epcs_1500_I_QP_27_240x240_CosineDecay_20220505-154218')
model_32768 = tf.keras.models.load_model('model_save_checkpoints_wavelets_haar_Lmbd_32768_epcs_1500_I_QP_27_240x240_CosineDecay_20220508-043447')


# Show the model architecture
# model.summary()
# write inputs to disk




Loading model


In [20]:
test(model_2048, p_on_test, out_decom, p_frame_out_bin)
test(model_4096, p_on_test, out_decom, p_frame_out_bin)
test(model_8192, p_on_test, out_decom, p_frame_out_bin)
test(model_16384, p_on_test, out_decom, p_frame_out_bin)
test(model_32768, p_on_test, out_decom, p_frame_out_bin)


compress
decompress
bin size:  2025 psnr:  37.32593128544646 BPP:  0.01171875
compress
decompress
bin size:  1486 psnr:  38.344064540817676 BPP:  0.008599537037037037
compress
decompress
bin size:  2895 psnr:  39.393952007766295 BPP:  0.01675347222222222
compress
decompress
bin size:  7428 psnr:  39.196075774892364 BPP:  0.042986111111111114
compress
decompress
bin size:  8813 psnr:  41.01333649927184 BPP:  0.051001157407407405


In [6]:
print("Loading model from saved checkpoints")
BATCH_SIZE = 1
EPOCHS = 1
STEPS_PER_EPOCH = 50
Height = 240
Width = 240
Channel = 3
NUM_FILTERS = 128
lmbda = 2**11 # 2048
lr_init = 1e-4
lr_alpha = 1e-8
early_stop = 800
I_QP=27
wavelet_name = "haar"
np_folder = "folder_cloud_test.npy"

checkponts_prev_path = "/workspaces/OpenDVCW/checkpoints_wavelets_haar_Lmbd_2048_epcs_1500_I_QP_27_240x240_CosineDecay_20220506-205303/"
model_2048_cp = TrainOpenDVCW(BATCH_SIZE, EPOCHS, STEPS_PER_EPOCH,
                              Height, Width, Channel, NUM_FILTERS,
                              lmbda, lr_init, lr_alpha, early_stop, I_QP,
                              wavelet_name, checkponts_prev_path, np_folder)
model_2048_cp.compile()
model_2048_cp.fit()
model_2048_cp.test(i_frame, p_frame, p_frame_out_bin, out_decom)
# model_2048_cp.check_psnr(i_frame, p_frame, p_frame_out_bin)

# model_2048 = tf.keras.models.load_model('model_save_checkpoints_wavelets_haar_Lmbd_2048_epcs_1500_I_QP_27_240x240_CosineDecay_20220506-205303')
# model_4096 = tf.keras.models.load_model('model_save_checkpoints_wavelets_haar_Lmbd_4096_epcs_1500_I_QP_27_240x240_CosineDecay_20220503-160827')
# model_8192 = tf.keras.models.load_model('model_save_checkpoints_wavelets_haar_Lmbd_8192_epcs_1500_I_QP_27_240x240_CosineDecay_20220504-144848')
# model_16384 = tf.keras.models.load_model('model_save_checkpoints_wavelets_haar_Lmbd_16384_epcs_1500_I_QP_27_240x240_CosineDecay_20220505-154218')
# model_32768 = tf.keras.models.load_model('model_save_checkpoints_wavelets_haar_Lmbd_32768_epcs_1500_I_QP_27_240x240_CosineDecay_20220508-043447')


Loading model from saved checkpoints


* [Model compiled]...
Loading weights


2022-05-28 14:41:04.216116: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-05-28 14:41:33.683475: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100


50/50 [==============================] - ETA: 0s - loss: 0.6846 - bpp: 0.3629 - mse: 1.5710e-04
Epoch 1: loss improved from inf to 0.68461, saving model to checkpoints_wavelets_haar_Lmbd_2048_epcs_1_I_QP_27_240x240_CosineDecay_20220528-144055/
50/50 [==============================] - 41s 151ms/step - loss: 0.6846 - bpp: 0.3629 - mse: 1.5710e-04
compress
in the compress
decompress
in decompress
bin size:  960 psnr:  37.764514655057496 bpp:  0.005555555555555556


In [16]:
# h264 test
p_frame_h264_decoded = "/workspaces/OpenDVCW/Test_com/h264/1.4k/decoded_h264_2.png"
h264_bin_path = "/workspaces/OpenDVCW/Test_com/h264/1.4k/frame1.h264"
compare(p_on_test, p_frame_h264_decoded, h264_bin_path)

# h264 test
p_frame_h264_decoded = "/workspaces/OpenDVCW/Test_com/h264/2.4k/decoded_h264_2.png"
h264_bin_path = "/workspaces/OpenDVCW/Test_com/h264/2.4k/frame1.h264"
compare(p_on_test, p_frame_h264_decoded, h264_bin_path)

# h264 test
p_frame_h264_decoded = "/workspaces/OpenDVCW/Test_com/h264/3.3k/decoded_h264_2.png"
h264_bin_path = "/workspaces/OpenDVCW/Test_com/h264/3.3k/frame1.h264"
compare(p_on_test, p_frame_h264_decoded, h264_bin_path)

# h264 test
p_frame_h264_decoded = "/workspaces/OpenDVCW/Test_com/h264/4.3k/decoded_h264_2.png"
h264_bin_path = "/workspaces/OpenDVCW/Test_com/h264/4.3k/frame1.h264"
compare(p_on_test, p_frame_h264_decoded, h264_bin_path)

# h264 test
p_frame_h264_decoded = "/workspaces/OpenDVCW/Test_com/h264/7.1k/decoded_h264_2.png"
h264_bin_path = "/workspaces/OpenDVCW/Test_com/h264/7.1k/frame1.h264"
compare(p_on_test, p_frame_h264_decoded, h264_bin_path)

# h264 test
p_frame_h264_decoded = "/workspaces/OpenDVCW/Test_com/h264/8.7k/decoded_h264_2.png"
h264_bin_path = "/workspaces/OpenDVCW/Test_com/h264/8.7k/frame1.h264"
compare(p_on_test, p_frame_h264_decoded, h264_bin_path)


bin size:  1452 psnr:  36.84876363907728 BPP:  0.008402777777777778
bin size:  2474 psnr:  37.254661164464146 BPP:  0.01431712962962963
bin size:  3379 psnr:  37.47398441432867 BPP:  0.019554398148148147
bin size:  4381 psnr:  37.744298474875755 BPP:  0.02535300925925926
bin size:  7157 psnr:  38.25742859922432 BPP:  0.04141782407407407
bin size:  8748 psnr:  38.450992768465305 BPP:  0.050625


In [18]:
# h265 test
p_frame_h265_decoded = "/workspaces/OpenDVCW/Test_com/h265/1.4k/decoded_h265_2.png"
h265_bin_path = "/workspaces/OpenDVCW/Test_com/h265/1.4k/frame1.h265"
compare(p_on_test, p_frame_h265_decoded, h265_bin_path)

# h265 test
p_frame_h265_decoded = "/workspaces/OpenDVCW/Test_com/h265/2.2k/decoded_h265_2.png"
h265_bin_path = "/workspaces/OpenDVCW/Test_com/h265/2.2k/frame1.h265"
compare(p_on_test, p_frame_h265_decoded, h265_bin_path)

# h265 test
p_frame_h265_decoded = "/workspaces/OpenDVCW/Test_com/h265/3.3k/decoded_h265_2.png"
h265_bin_path = "/workspaces/OpenDVCW/Test_com/h265/3.3k/frame1.h265"
compare(p_on_test, p_frame_h265_decoded, h265_bin_path)

# h265 test
p_frame_h265_decoded = "/workspaces/OpenDVCW/Test_com/h265/4.3k/decoded_h265_2.png"
h265_bin_path = "/workspaces/OpenDVCW/Test_com/h265/4.3k/frame1.h265"
compare(p_on_test, p_frame_h265_decoded, h265_bin_path)

# h265 test
p_frame_h265_decoded = "/workspaces/OpenDVCW/Test_com/h265/7.4k/decoded_h265_2.png"
h265_bin_path = "/workspaces/OpenDVCW/Test_com/h265/7.4k/frame1.h265"
compare(p_on_test, p_frame_h265_decoded, h265_bin_path)

# h265 test
p_frame_h265_decoded = "/workspaces/OpenDVCW/Test_com/h265/8.7k/decoded_h265_2.png"
h265_bin_path = "/workspaces/OpenDVCW/Test_com/h265/8.7k/frame1.h265"
compare(p_on_test, p_frame_h265_decoded, h265_bin_path)

bin size:  1451 psnr:  37.635756498430204 BPP:  0.008396990740740741
bin size:  2195 psnr:  37.958438318963836 BPP:  0.012702546296296297
bin size:  3351 psnr:  38.23862877567929 BPP:  0.01939236111111111
bin size:  4329 psnr:  38.47398597765003 BPP:  0.025052083333333332
bin size:  7405 psnr:  38.80474786529784 BPP:  0.04285300925925926
bin size:  8713 psnr:  38.88156179902449 BPP:  0.050422453703703705
